In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install mediapipe

Note: you may need to restart the kernel to use updated packages.


In [7]:
import cv2
import time
import winsound
import mediapipe as mp

In [8]:
# Alarm helper function
def beep_alarm():
    duration = 500  # milliseconds
    freq = 1000     # Hz
    winsound.Beep(freq, duration)

In [9]:
# Function to close the camera
def close_camera(cam, out):
    """Release the camera and close all OpenCV windows."""
    cam.release()
    out.release()
    cv2.destroyAllWindows()
    print("Camera closed successfully.")

In [10]:
def main():
    # Initialize Mediapipe face mesh
    mp_face_mesh = mp.solutions.face_mesh
    face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True)
    mp_drawing = mp.solutions.drawing_utils

    # Open the default camera
    cam = cv2.VideoCapture(0)

    # Get the default frame width and height
    frame_width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (frame_width, frame_height))

    # Time trackers
    head_left_start = None
    head_right_start = None
    eyes_closed_start = None

    while True:
        ret, frame = cam.read()
        if not ret:
            close_camera(cam, out)
            break

        # Convert frame to RGB for mediapipe
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(rgb_frame)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # Draw face landmarks
                mp_drawing.draw_landmarks(frame, face_landmarks, mp_face_mesh.FACEMESH_CONTOURS)

                # Extract landmarks: nose tip, eyes
                h, w, _ = frame.shape
                nose = face_landmarks.landmark[1]
                left_eye = face_landmarks.landmark[159]
                right_eye = face_landmarks.landmark[386]
                left_eye_bottom = face_landmarks.landmark[145]
                right_eye_bottom = face_landmarks.landmark[374]

                nose_x = int(nose.x * w)

                # Head left/right detection
                center_x = w // 2
                tolerance = 50

                if nose_x < center_x - tolerance:
                    if head_left_start is None:
                        head_left_start = time.time()
                    elif time.time() - head_left_start > 3:
                        beep_alarm()
                else:
                    head_left_start = None

                if nose_x > center_x + tolerance:
                    if head_right_start is None:
                        head_right_start = time.time()
                    elif time.time() - head_right_start > 3:
                        beep_alarm()
                else:
                    head_right_start = None

                # Eye closure detection
                left_eye_height = abs(left_eye.y - left_eye_bottom.y)
                right_eye_height = abs(right_eye.y - right_eye_bottom.y)

                if left_eye_height < 0.01 and right_eye_height < 0.01:
                    if eyes_closed_start is None:
                        eyes_closed_start = time.time()
                    elif time.time() - eyes_closed_start > 3:
                        beep_alarm()
                else:
                    eyes_closed_start = None

        # Write the frame to the output file
        out.write(frame)

        # Display the captured frame
        cv2.imshow('Camera', frame)

        # Press 'q' to exit
        if cv2.waitKey(1) == ord('q'):
            close_camera(cam, out)
            break

In [ ]:
# Run the main function
if __name__ == "__main__":
    main()

C:\Users\saqib\anaconda3\envs\mediapipe_env\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
